In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("../Data/courses_ratings.csv", index_col=['course_id'])
#df.head()

In [3]:
df_courses = pd.read_csv("../Data/courses_dataset.csv")
#df_courses.head()

In [4]:
df = df.fillna(0).astype(int)
#df.head()

In [5]:
from sklearn.neighbors import NearestNeighbors

In [6]:
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(df.values)
distances, indices = knn.kneighbors(df.values, n_neighbors=3)

In [7]:
# indices

indices shows the nearest courses to each course. Each row corresponds to the row in the df. The first element in a row is the most similar (nearest) course. It is the course itself. The second element is the second nearest, and the third is the third nearest. For example, in the first row [0,7,5], the nearest course to 'course_0' is itself, the second nearest course is 'course_7', and the third is 'course_5'.

In [8]:
# distances

In [9]:
# print(indices[21])

distances shows the distance between courses. Each number in this matrix corresponds to the number in the indices matrix.

In [10]:
for title in df.index:
    index_user_likes = df.index.tolist().index(title) # get an index for a course
    
    print("Index: ", index_user_likes)
    
    
    sim_courses = indices[index_user_likes].tolist() # make list for similar courses
    
    print("Similar cources: ", sim_courses)
    
    course_distances = distances[index_user_likes].tolist() # the list for distances of similar courses
    
    print("Distances cources: ", course_distances) 
    
    id_course = sim_courses.index(index_user_likes) # get the position of the course itself in indices and distances
    
    print("ID cources: ", id_course)

    #print('Similar courses to '+str(df.index[index_user_likes])+':\n')


    sim_courses.remove(index_user_likes) # remove the course itself in indices
    course_distances.pop(id_course) # remove the course itself in distances
    
    
    j = 1
    
    
    print(sim_courses)
    
    
    
    
    for i in sim_courses:
       print(str(j)+': '+str(df.index[i])+', the distance with '+str(title)+': '+str(course_distances[j-1]))
       j = j + 1

    print('\n')
    

Index:  0
Similar cources:  [0, 2917, 785]
Distances cources:  [0.0, 0.07852140175826994, 0.08233706451775302]
ID cources:  0
[2917, 785]
1: 584758, the distance with 1070968: 0.07852140175826994
2: 1233302, the distance with 1070968: 0.08233706451775302


Index:  1
Similar cources:  [1, 1237, 614]
Distances cources:  [0.0, 0.023532708129441082, 0.031504003041813666]
ID cources:  0
[1237, 614]
1: 177614, the distance with 1113822: 0.023532708129441082
2: 557436, the distance with 1113822: 0.031504003041813666


Index:  2
Similar cources:  [2, 143, 2374]
Distances cources:  [2.220446049250313e-16, 2.220446049250313e-16, 2.220446049250313e-16]
ID cources:  0
[143, 2374]
1: 48841, the distance with 1006314: 2.220446049250313e-16
2: 320238, the distance with 1006314: 2.220446049250313e-16


Index:  3
Similar cources:  [3, 1632, 3637]
Distances cources:  [2.220446049250313e-16, 0.029857499854668124, 0.037466578120378125]
ID cources:  0
[1632, 3637]
1: 908664, the distance with 1210588: 0.02

ValueError: 20 is not in list

In [ ]:
# x = sim_courses.index(3)
# x

In [11]:
df1 = df.copy()

def recommend_courses(user, num_recommended_course):
    recommended_course = []

    for m in df[df[user] == 0].index.tolist():
        index_df = df.index.tolist().index(m)
        predicted_rating = df1.iloc[index_df, df1.columns.tolist().index(user)]
        recommended_course.append((m, predicted_rating))

    sorted_rm = sorted(recommended_course, key=lambda x:x[1], reverse=True)

    print('The list of the Recommended course \n')
    rank = 1
    recommandations = list()
    for recommended_course in sorted_rm[:num_recommended_course]:
        recommandations.append(recommended_course[0])
        print('{}: {} - predicted rating:{}'.format(rank, recommended_course[0], recommended_course[1]))
        rank = rank + 1
    return recommandations

In [13]:
recommendations = recommend_courses('Pooja', 5)

The list of the Recommended course 

1: 28295 - predicted rating:3
2: 185526 - predicted rating:2
3: 837322 - predicted rating:1
4: 1070968 - predicted rating:0
5: 1210588 - predicted rating:0


In [ ]:
#course_index = list()
#for recommendation in recommendations:
#    course_index.append(df_courses.index[df_courses["course_id"]==recommendation].tolist())

In [ ]:
#course_indexs = [item for sublist in course_index for item in sublist]
#course_indexs

In [ ]:
#df_courses[df_courses.course_id.isin(recommendations)]

In [14]:
df1 = df.copy()

def course_recommender(user, num_neighbors, num_recommendation):
    number_neighbors = num_neighbors
    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(df.values)
    distances, indices = knn.kneighbors(df.values, n_neighbors=number_neighbors)

    user_index = df.columns.tolist().index(user)

    for m,t in list(enumerate(df.index)):
        if df.iloc[m, user_index] == 0:
            sim_courses = indices[m].tolist()
            course_distances = distances[m].tolist()
            
            if m in sim_courses:
                id_course = sim_courses.index(m)
                sim_courses.remove(m)
                course_distances.pop(id_course) 

            else:
                sim_courses = sim_courses[:num_neighbors-1]
                course_distances = course_distances[:num_neighbors-1]

            course_similarity = [1-x for x in course_distances]
            course_similarity_copy = course_similarity.copy()
            nominator = 0

            for s in range(0, len(course_similarity)):
                if df.iloc[sim_courses[s], user_index] == 0:
                    if len(course_similarity_copy) == (number_neighbors - 1):
                        course_similarity_copy.pop(s)

                    else:
                        course_similarity_copy.pop(s-(len(course_similarity)-len(course_similarity_copy)))

                else:
                    nominator = nominator + course_similarity[s]*df.iloc[sim_courses[s],user_index]

            if len(course_similarity_copy) > 0:
                if sum(course_similarity_copy) > 0:
                    predicted_r = nominator/sum(course_similarity_copy)

                else:
                    predicted_r = 0

            else:
                predicted_r = 0

            df1.iloc[m,user_index] = predicted_r
    recommandations = recommend_courses(user, num_recommendation)

In [15]:
recommandations = course_recommender('Rohit', 5, 5)
#print("recommandations", recommandations)

The list of the Recommended course 

1: 312860 - predicted rating:3.0000000000000004
2: 147754 - predicted rating:3.0000000000000004
3: 151956 - predicted rating:3.0000000000000004
4: 94432 - predicted rating:3.0000000000000004
5: 640062 - predicted rating:3.0
